In [1]:
import numpy as np
from scipy.interpolate import splrep, splev
import sys
sys.path.insert(0,'/Volumes/Transcend/software/AIMD')
from fingerprint import fingerprint
from rules import compile_rules
from smoothingkernel import *

import ase.io
from ase import Atoms

In [2]:
rc = 6.0
rules = compile_rules( '''
title: LJGaussianWildcard
preamble:
    - from numpy import exp
    - from smoothingkernel import *
env:
    - rc
elements:
    '*':
        density: 'lambda rij, rjk, exp = exp, wd = Tent( 3, rc, normalize = False ): wd(rij) * exp( -0.5 * rjk**2 / ( 1.5 + rij * 0.25 )**2 ) / ( 1.5 + rij * 0.25 )'
        #density: 'lambda rij, rjk, exp = exp: exp( -0.5 * rjk**2 / ( 1.5 + rij * 0.25 )**2 ) / ( 1.5 + rij * 0.25 )'
        gravity: 'lambda rij: 1.0'
''', rc = rc )

In [3]:
descr = fingerprint('3-(14,26,38)', rc, 3, [14, 26, 38], rules, weight_int = Tent(5, rc))

In [11]:
descr.info

<bound method FingerprintBase.info of <class 'fingerprint.Fingerprint_50de4af5db533a4e320d5e6fbceeb441e9751502'>>

In [4]:
atoms = Atoms( "N2", [[0,0,0],[2,0,0]] )

In [5]:
rmin = 1.0
rmax = rc
n_sample = int(( rmax - rmin ) // 0.1 + 1)
print(n_sample)

50


In [6]:
fingerprints = []
numbers = np.linspace(rmin, rmax, n_sample)

In [7]:
forces_a1 = []
forces_a2 = []
for item in numbers:
    forces_a1.append([2*item - 4, 0, 0])
    forces_a2.append([-2*item - 4, 0, 0])
forces_N2 = [forces_a1, forces_a2]

In [8]:
for dr in numbers:
    atoms.set_positions( [[0,0,0],[dr,0,0]] )
    fingerprints.append( descr.extract_one( atoms, 0, force = False ) )
    fingerprints[-1].set( y = forces_N2 )

In [9]:
dist = np.array( [ [ f1 - f2 for f2 in fingerprints ] for f1 in fingerprints ] )
dist = dist / dist.max()